In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import argparse
from scipy.stats import gaussian_kde
import os
import yaml
import h5py
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import time
import warnings

In [ ]:
import sys
sys.path.append(r'../tools_F-J/')
sys.path.append(r'../tools_F-J/toollib_DisbaCode')
from objective_function import (ObjectiveFunctionDerivativeUsed, Forward)
from toollib_DisbaCode import forward
from toollib_DisbaCode import objective_function
from toollib_DisbaCode import empirical_relation
from toollib_DisbaCode import plot_disp
from toollib_DisbaCode import plot_model
from toollib_DisbaCode import plot_lcurve
from toollib_DisbaCode import plot_inversion    
from toollib_DisbaCode import plot_kernel

#from toollib_standard import maplib
#from toollib_standard import mathlib
#from toollib_standard import filelib
#from toollib_standard import stacklib
from toollib_standard import plotlib

In [ ]:
flag_project = 1 # 0--regular; 1--repartrition; 2--voronoi
flag_mode = 2 # 0--fundamental; 1--Overtones; 2--Both
flag_forward = 1 # 0--no forward calculation; 1--forward calculation
file_init = 'initial/initial_model_4.txt'

In [ ]:
if flag_project == 0:
    file_project = 'a-project.yml'
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
elif flag_project == 2:
    file_project = 'a-project_voro.yml'
    
with open(file_project, 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']

#name_project = 'project/output_FJSJ_16-01/'               
name_project = 'project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/'         
#name_project = 'project_voronoi/voronoi_01-03/'         

In [ ]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

In [ ]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [ ]:
dir_ds = dir_project + info_basic['rdir_ds']
dir_partition = dir_project + info_basic['rdir_partition']
dir_inv = dir_project + info_basic['rdir_inv_BFGS']
f0 = info_basic_bi['f']
c = np.linspace(info_basic['fj_c_min'], info_basic['fj_c_max'], info_basic['fj_c_num'])

In [ ]:
flag_near = 15

In [ ]:
#filename = dir_project+dir_inv_dispernet+ 'inv3.txt'
#nums = np.loadtxt(filename,dtype=int)
nums = [1] 
key_subworks_all = info_basic['key_subworks']
key_subworks = []
for num in nums:
    for key_subwork in key_subworks_all:
        if str(num) == key_subwork.split('--')[0]:
            key_subworks.append(key_subwork)
            break
#key_subworks = info_basic['key_subworks']
key_subworks

In [ ]:
file_config_fund = 'config_inv_fund.yml'
file_config = 'config_inv.yml'
with open(dir_inv+file_config, 'r') as fp:
    config = yaml.safe_load(fp)
with open(dir_inv+file_config_fund, 'r') as fp:
    config_fund = yaml.safe_load(fp)

In [ ]:

dir_image = dir_project +  info_basic['rdir_image'] + 'inversion_compare_near/'
if not os.path.exists(dir_image):
    os.mkdir(dir_image)

info_basic['rdir_BFGS_invfile'] = info_basic['rdir_inv_BFGS'] + 'inversion/'
info_basic['rdir_disp_model'] = info_basic['rdir_inv_BFGS'] + 'disp_model/'
info_basic['rdir_model'] = info_basic['rdir_inv_BFGS'] + 'model/'
dir_invfile = dir_project + info_basic['rdir_BFGS_invfile']
dir_disp_model = dir_project + info_basic['rdir_disp_model']
dir_model = dir_project + info_basic['rdir_model']

if not os.path.exists(dir_invfile):
    os.mkdir(dir_invfile)
if not os.path.exists(dir_disp_model):
    os.mkdir(dir_disp_model)
if not os.path.exists(dir_model):
    os.mkdir(dir_model)
info_basic['rdir_BFGS_invfile_fund'] = info_basic['rdir_inv_BFGS'] + 'inversion_fund/'
info_basic['rdir_disp_model_fund'] = info_basic['rdir_inv_BFGS'] + 'disp_model_fund/'
info_basic['rdir_model_fund'] = info_basic['rdir_inv_BFGS'] + 'model_fund/'
dir_invfile_fund = dir_project + info_basic['rdir_BFGS_invfile_fund']
dir_disp_model_fund = dir_project + info_basic['rdir_disp_model_fund']
dir_model_fund = dir_project + info_basic['rdir_model_fund']
if not os.path.exists(dir_invfile_fund):
    os.mkdir(dir_invfile_fund)
if not os.path.exists(dir_disp_model_fund):
    os.mkdir(dir_disp_model_fund)
if not os.path.exists(dir_model_fund):
    os.mkdir(dir_model_fund)

### Read data

In [ ]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
stalist_all = stainfo['Station'].tolist() 
lat_stations_all =  stainfo['latitude'].tolist() 
lon_stations_all =  stainfo['longitude'].tolist() 

In [ ]:
# fault surface trace
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

In [ ]:
stations_partition = {}
lat_stations_partition = {}
lon_stations_partition = {}
lat_centroid_partition = []
lon_centroid_partition = []
for key in info_basic['key_subworks']:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations_partition[key] = stations_this
    lat_stations_partition[key] = lat_stations_this.astype(float)
    lon_stations_partition[key] = lon_stations_this.astype(float)
    lat_centroid_partition.append(np.mean(lat_stations_this.astype(float)))
    lon_centroid_partition.append(np.mean(lon_stations_this.astype(float)))

In [ ]:
def compute_affine_transform(original_points, target_points):
    A_matrix = np.array([[original_points[0][0], original_points[0][1], 1, 0, 0, 0],
                         [0, 0, 0, original_points[0][0], original_points[0][1], 1],
                         [original_points[1][0], original_points[1][1], 1, 0, 0, 0],
                         [0, 0, 0, original_points[1][0], original_points[1][1], 1],
                         [original_points[2][0], original_points[2][1], 1, 0, 0, 0],
                         [0, 0, 0, original_points[2][0], original_points[2][1], 1]])

    A1_B1_C1 = np.array([target_points[0][0], target_points[0][1], target_points[1][0], target_points[1][1], target_points[2][0], target_points[2][1]])

    coefficients = np.linalg.solve(A_matrix, A1_B1_C1)

    affine_matrix = np.array([[coefficients[0], coefficients[1], coefficients[2]],
                               [coefficients[3], coefficients[4], coefficients[5]],
                               [0, 0, 1]])

    return affine_matrix

In [ ]:
# Affine transformation
lon_stations_all_new = []
lat_stations_all_new = []
for sta in stalist_all:
    if int(sta[1:3]) <= 60:
        lon_stations_all_new.append(lon_stations_all[stalist_all.index(sta)])
        lat_stations_all_new.append(lat_stations_all[stalist_all.index(sta)])
refs = ['R0101','R6001','R6020']
lon_refs = [lon_stations_all[stalist_all.index(ref)] for ref in refs]
lat_refs = [lat_stations_all[stalist_all.index(ref)] for ref in refs]
loc_refs = np.column_stack([lon_refs,lat_refs])
loc_refs_new = np.array([[0,0],[600,0],[600,600]])

affine_matrix = compute_affine_transform(loc_refs, loc_refs_new)

In [ ]:
# all stations
x_stations_all = []
y_stations_all = []
for i in range(len(lon_stations_all)):
    loc_sta = np.array([lon_stations_all[i],lat_stations_all[i],1])
    loc_sta_new = np.dot(affine_matrix,loc_sta)
    x_stations_all.append(loc_sta_new[0])
    y_stations_all.append(loc_sta_new[1])
x_centroid_partition = []
y_centroid_partition = []
for i in range(len(lon_centroid_partition)):
    loc_centroid = np.array([lon_centroid_partition[i],lat_centroid_partition[i],1])
    loc_centroid_new = np.dot(affine_matrix,loc_centroid)
    x_centroid_partition.append(loc_centroid_new[0])
    y_centroid_partition.append(loc_centroid_new[1])

In [ ]:
ds_all = {}
for key in key_subworks:
    print(key+ ' '+str(key_subworks.index(key)+1) + '/'+str(len(key_subworks)))
    ds = h5py.File(dir_ds+'ds_'+key+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]
    ds_all[key] = plotlib.smooth_ds(ds_remove)

In [ ]:
def plot_fj(ax,ds_linear,title0,f,c,index,v_min,v_max,c_map):
    ax.imshow(np.flip(ds_linear,0),aspect='auto',extent=[min(f),max(f),min(c),max(c)],vmin=v_min,vmax = v_max, cmap = c_map)
    #plt.imshow(np.flip(ds_linear,0),extent=[min(f),max(f),min(c),max(c)],aspect='auto',cmap='jet',vmin=0,vmax=1)
    #ax.pcolormesh(f,c,ds_linear,cmap='jet',vmin=0,vmax=1)
    if index == 0:
        ax.set_title(title0)
    else:
        ax.set_title('('+chr(96+index)+')',loc='left')
    ax.set_xlabel('Normalized Frequency/ Hz')
    ax.set_ylabel('Velocity/ m/s')
    #ax.set_xlim(xlim)
    return ax

### Forard and plot estimated disp and Vs

In [ ]:
def find_nearest(key_subwork):
    global key_subworks_all
    global x_centroid_partition
    global y_centroid_partition
    global lon_centroid_partition
    global lat_centroid_partition
    global flag_near

    y_centroid = y_centroid_partition[key_subworks_all.index(key_subwork)]
    x_centroid = x_centroid_partition[key_subworks_all.index(key_subwork)]
    dist = np.sqrt((np.array(y_centroid_partition)-y_centroid)**2 + (np.array(x_centroid_partition)-x_centroid)**2)
    dist_sort = np.sort(dist)
    key_nearest = []
    lon_nearest = []
    lat_nearest = []
    dist_nearst = []
    for i in range(len(dist_sort)):
        if dist_sort[i] < flag_near and dist[i] != 0:
            index = np.where(dist==dist_sort[i])[0][0]
            key_this = key_subworks_all[index]
            key_nearest.append(key_subworks_all[key_subworks_all.index(key_this)])
            lon_nearest.append(lon_centroid_partition[key_subworks_all.index(key_this)])
            lat_nearest.append(lat_centroid_partition[key_subworks_all.index(key_this)])
            dist_nearst.append(dist_sort[i])


    return key_nearest, lon_nearest, lat_nearest,dist_sort

In [ ]:
def plot_model_near(config,dir_model,key_subwork,key_nearest,dist_nearst,ax):
    global flag_near
    config_plot = config['plot']
    zmax = config_plot['zmax']
    vsmin, vsmax = config_plot['vs_lim']
    model_init = np.loadtxt(dir_inv + config['model_init']+'.txt')
    z = model_init[:, 1]
    vs_init = model_init[:, 3]
    unit = 'km'
    km2m = 1
    hw = config['init_half_width']
    v1, v2 = vs_init - hw, vs_init + hw
    vs1_plot = np.append(v1, v1[-1])
    vs2_plot = np.append(v2, v2[-1])
    z_plot = np.append(z, zmax) * km2m

    ax.step(vs_init,z,c = 'k',label = 'initial',linewidth=0.8)
    ax.set_xlim([vsmin, vsmax])
    ax.set_ylim([0, zmax * km2m])
    ax.set_xlabel('Vs (km/s)')
    ax.set_ylabel('Depth ({:s})'.format(unit))
    ax.invert_yaxis()
    ax.step(vs1_plot,z_plot,'--',c='gray',alpha=0.8)
    ax.step(vs2_plot, z_plot, '--', c='gray', alpha=0.8)

    data = np.loadtxt(dir_model + 'model_' + key_subwork + '.txt')
    z = data[:, 1]
    z = np.append(z, [zmax,]) * km2m
    vs = data[:, 3]
    vs = np.append(vs, [vs[-1],])
    p3, = ax.step(vs,z,'-',c='b',alpha=0.6,linewidth=4,label = key_subwork)

    #cs = ['b','plum','y','g','cyan','lime']
    #cmap = plt.cm.get_cmap('coolwarm')
    cmap = matplotlib.colormaps['afmhot']
    norm = plt.Normalize(10, flag_near)
    colors = [cmap(norm(value)) for value in dist_nearst]
    for i in range(len(key_nearest)):
        key = key_nearest[i]
        data = np.loadtxt(dir_model + 'model_' + key + '.txt')
        z = data[:, 1]
        z = np.append(z, [zmax,]) * km2m
        vs = data[:, 3]
        vs = np.append(vs, [vs[-1],])
        p3, = ax.step(vs,z,'-',c=colors[i],alpha=0.6,linewidth=2,label = key)
    ax.legend(loc='lower left')
    sm = ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])
    cbar = plt.colorbar(sm,ax = ax)
    return ax

In [ ]:
c_map = 'jet'
v_max = None
v_min = 0.1

all_disp = 0
show_std = 1
show_init = 0
plot_fund = 1
xlim_f = [0.5,30]
xlim_T = [-1,1]
ylim_c = [0.25,1.8]
clim = [0,1]
fmin = 1
fmax = 30
cmin = 0.1
cmax = 2.5
vmax = 0.3

max_mode = 4

In [ ]:
flag_num = 0
inter_forward = 5
linestyle = ['*','o','^','+','v','<','>','p','h']
show_init = 0
for key_subwork in key_subworks:
    flag_num += 1
    print(key_subwork + ', '+str(key_subworks.index(key_subwork)+1) + '/' + str(len(key_subworks)) ) 

    # configuration
    file_inv =  'ds_'+key_subwork+'curve'
    file_disp_data = dir_project + info_basic['rdir_disp_pick']+'ds_'+key_subwork+'curve.txt'
    file_model = 'model_'+key_subwork+'.txt'
    file_disp_forward = dir_disp_model +  'disp_model_'+key_subwork+'.txt'
    if not os.path.exists(dir_invfile+file_inv):
        print(key_subwork+'Overtone inversion data file not exist')
        plt.close()
        continue
    file_model_fund = 'model_'+key_subwork+'.txt'
    file_disp_forward_fund = dir_disp_model_fund +  'disp_model_'+key_subwork+'.txt'
    if not os.path.exists(dir_invfile_fund+file_inv):
        print(key_subwork+' Fundamental inversion data file not exist')
        plt.close()
        continue    
    
    fig = plt.figure(figsize=(24, 16))
    # plot partition
    stalist = stations_partition[key_subwork]
    lat_stations_this = lat_stations_partition[key_subwork]
    lon_stations_this = lon_stations_partition[key_subwork]
    ax_partition = fig.add_subplot(231)
    ax_partition=plotlib.plot_area(ax_partition,lon_stations_all,lat_stations_all,lon_stations_this,lat_stations_this,markersize = 0.75 ,markersize2 = 3)
    ax_partition.scatter(lon_centroid_partition,lat_centroid_partition,marker='^',color='y',s=4)
    for i in range(len(faults)):
        ax_partition.plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k--',lw = 0.5)
    ax_partition.set_title('Partition')
    key_nearest, lon_nearest, lat_nearest, dist_nearst = find_nearest(key_subwork)
    cmap = matplotlib.colormaps['winter']
    norm = plt.Normalize(10, flag_near)
    colors = [cmap(norm(value)) for value in dist_nearst]
    for i in range(len(key_nearest)): 
        ax_partition.scatter(lon_nearest[i],lat_nearest[i],marker='o',color=colors[i],s=40,label = key_nearest[i])
    sm = ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])
    cbar = plt.colorbar(sm,ax = ax_partition, fraction = 0.01,pad=0.01)
    ax_partition.scatter(np.mean(lon_stations_this),np.mean(lat_stations_this),marker='^',color='b',s=100,label = key_subwork)
    ax_partition.legend()
    stations_nearst = set()
    lon_stations_nearest = []
    lat_stations_nearest = []
    for key_subwork_this in key_nearest:
        stations_nearst.update(stations_partition[key_subwork_this])
    for sta in stations_nearst:
        lon_stations_nearest.append(lon_stations_all[stalist_all.index(sta)])
        lat_stations_nearest.append(lat_stations_all[stalist_all.index(sta)])
    lon_stations_nearest = list(lon_stations_nearest)
    lat_stations_nearest = list(lat_stations_nearest)
    ax_partition.scatter(lon_stations_nearest,lat_stations_nearest,marker='o',color='b',s=10)
    ax_partition.set_aspect('equal')


    title0 = "FJ with forward disp curve"
    ax_FJ = fig.add_subplot(2,3,2)
    ax_FJ.set_xlim(xlim_f)
    ax_FJ.set_ylim(ylim_c)
    ds_remove = ds_all[key_subwork]
    ax_FJ = plotlib.plot_fj(ax_FJ,ds_remove,title0,f0,c,0,v_min = v_min,v_max=v_max,c_map=c_map)
    disp_data = np.loadtxt(file_disp_data)
    num_m_data = len(set(disp_data[:,2]))
    modes_data = set(disp_data[:,2].astype(int))
        # data
    for mode in modes_data:
        data_mode = disp_data[disp_data[:, 2] == mode]
        if mode==0:
            ax_FJ.plot(data_mode[:, 0], data_mode[:, 1], 'k^',label='data',linewidth=4)
        else:
            ax_FJ.plot(data_mode[:, 0], data_mode[:, 1],  'k^',linewidth=4 )
        # foward
    file_disp_forward = dir_disp_model +  'disp_model_'+key_subwork+'.txt'
    disp_forward = np.loadtxt(file_disp_forward)
    num_m = len(set(disp_forward[:,2]))
    modes = set(disp_forward[:,2].astype(int))
    for mode in modes:
        disp_forward_mode = disp_forward[disp_forward[:, 2] == mode]
        if mode==0:
            ax_FJ.scatter(disp_forward_mode[::inter_forward, 0], disp_forward_mode[::inter_forward, 1], color='greenyellow',marker = linestyle[mode],label='Overtone estemate',s=10)
        else:
            ax_FJ.scatter(disp_forward_mode[::inter_forward, 0], disp_forward_mode[::inter_forward, 1], color='greenyellow',marker = linestyle[mode],s=10)
    file_disp_forward_fund = dir_disp_model_fund +  'disp_model_'+key_subwork+'.txt'
    disp_forward_fund = np.loadtxt(file_disp_forward_fund)
    num_m = len(set(disp_forward_fund[:,2]))
    modes = set(disp_forward_fund[:,2].astype(int))
    for mode in modes:
        disp_forward_fund_mode = disp_forward_fund[disp_forward_fund[:, 2] == mode]
        if mode==0:
            ax_FJ.scatter(disp_forward_fund_mode[::inter_forward, 0], disp_forward_fund_mode[::inter_forward, 1], color='red',marker = linestyle[mode],label='Fundamental estemate',s=10)
        else:
            ax_FJ.scatter(disp_forward_fund_mode[::inter_forward, 0], disp_forward_fund_mode[::inter_forward, 1], color='red',marker= linestyle[mode],s=10)


    # plot fund and over
    ax_vs = fig.add_subplot(234)
    ax_vs,model_fund,fmt = plot_inversion.plot_model(config, file_inv, show_init, file_model,file_init,ax_vs,file_model_fund =dir_model_fund + file_model_fund,plot_fund=1,dir_this = dir_inv)
    std_fund = model_fund[:,5]
    z_step = model_fund[:,1]
    
    # plot fund velocity
    ax_fund = fig.add_subplot(235)
    ax_fund = plot_model_near(config_fund,dir_model_fund,key_subwork,key_nearest,dist_nearst,ax_fund)
    ax_over = fig.add_subplot(236)
    ax_over = plot_model_near(config,dir_model,key_subwork,key_nearest,dist_nearst,ax_over)

    plt.tight_layout()
    plt.savefig(dir_image+key_subwork+'_compare.png')
    plt.close()